In [1]:
import os
import sys
import pandas as pd
import numpy as np
import datetime as dt

In [2]:
# Paths, input, and output file names

outer_dir = os.path.abspath(os.path.join(os.getcwd() ,"../.."))
refpth = os.path.join(outer_dir, 'IFT_files', 'Reference Files')
sfe_char_csv = os.path.join(refpth, 'TNC SFE LOI Characteristics with MAF.csv')
sfe_char_xls= os.path.join(refpth, 'TNC SFE LOI Characteristics.xlsx')
subset_xls = os.path.join(refpth, 'TNC Subset SFE LOIs.xlsx')



startdir = os.path.join(outer_dir, 'IFT_files', 'IFT Results')
mpof1 = os.path.join(startdir, "All LOI MPOF - Default IFTs.csv")
mpof2 = os.path.join(startdir, "MPOF Variants", "All LOI MPOF - 20% of 10th Percentile Hydrograph, Daily IFTs.csv")
mpof1_start_month = 4
mpof1_start_day = 1
mpof1_end_month = 12
mpof1_end_day = 14
out_csv = os.path.join(startdir, 'All LOI MMPOF_TNC.csv')

In [3]:
# Set the date range from the mpof1 time series
def ts_sub(df):
    """Adds day, month, and a mpof1 column to the data frame to identify a time range"""
    df['month'] = pd.DatetimeIndex(df['TS']).month
    df['day'] = pd.DatetimeIndex(df['TS']).day
    df.loc[(df['month'] == mpof1_end_month) & (df['day'] <= mpof1_end_day), 'mpof1'] = 1
    df.loc[(df['month'] == mpof1_start_month) & (df['day'] >= mpof1_start_day), 'mpof1'] = 1
    df.loc[(df['month'] > mpof1_start_month) & (df['month'] < mpof1_end_month), 'mpof1'] = 1
    return df

In [4]:
df1 = pd.read_csv(mpof1)
df1['TS'] = pd.to_datetime(df1['TS'])

In [5]:
df2 = pd.read_csv(mpof2)
df2['TS'] = pd.to_datetime(df2['TS'])

In [6]:
df1 = ts_sub(df1)
df2 = ts_sub(df2)

In [7]:
# append together
df1_sub = df1.loc[df1.mpof1 == 1].copy()
df2_sub = df2.loc[~(df2.mpof1 == 1)].copy()
df = df1_sub.append(df2_sub, ignore_index=True)
df = df.sort_values(by=['TS'], ascending=[True])
df = df.drop(['month', 'day', 'mpof1'], axis=1)

In [8]:
print("df1 has {} records".format(len(df1)))
print("df2 has {} records".format(len(df2)))
print("df has {} records".format(len(df)))

df1 has 25933 records
df2 has 25933 records
df has 25933 records


In [9]:
df.to_csv(out_csv, index=False)